In [ ]:
import pickle
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

In [ ]:
data_path = "./deeplearning_results/"
dl_results = os.listdir(data_path)
dl_results_paths = [data_path + file for file in dl_results]

In [ ]:
results = []
i = 0
for path in dl_results_paths:
    temp = {}
    if "lstm" in path:
        temp['name'] = "BiLSTM"
        in_size = dl_results[i].split("_")[2].replace('in', '')
        out_size = dl_results[i].split("_")[3].replace('out', '')
        
    else:
        temp['name'] = "MLP"
        in_size = dl_results[i].split("_")[1].replace('in', '')
        out_size = dl_results[i].split("_")[2].replace('out', '')
    
    with open(path, "rb") as f:
        payload = pickle.load(f)
    
    
    temp['INPUT SIZE'] = in_size
    temp['OUTPUT SIZE'] = out_size
    temp['TARGET'] = dl_results[i].split("_")[-1].replace('target', '').replace('.pkl','')
    temp['MAPE'] = payload['MAPE']
    temp['SMAPE'] = payload['sMAPE']
    temp['MAE'] = payload['MAE']
    temp['MDA'] = payload['MDA']

    results.append(temp)
    i = i + 1

df = pd.DataFrame(results)#.sort_values([], ascending=True).reset_index(drop=True)

In [ ]:
df[df['name'] == 'MLP']

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

def mda(y_true, y_pred):
    return np.mean((np.sign(y_true[1:] - y_pred[:-1]) == np.sign(y_pred[1:] - y_pred[:-1]).astype(int)))

def calculate_metrics(y_real, y_pred):
    metrics = {
        "MAPE": mean_absolute_percentage_error(y_real, y_pred),
        "sMAPE": smape(y_real, y_pred),
        "MAE": mean_absolute_error(y_real, y_pred),
        "MDA": mda(y_real, y_pred)
    }
    return metrics

In [ ]:
data_path = "./results/"
ml_results = os.listdir(data_path)
ml_results_paths = [data_path + file for file in ml_results]

results = []
i = 0
for path in ml_results_paths:
    temp = {}
    with open(path, "rb") as f:
        payload = pickle.load(f)
    
    name = 'RFR' if ml_results[i].split('_')[1] == 'random' else ml_results[i].split('_')[1]
    in_size = ml_results[i].split('_')[-3].replace('in','')
    out_size = ml_results[i].split('_')[-2].replace('out','')
    target = ml_results[i].split('_')[-1].replace('.pkl', '').replace('t','')
    
    #metrics = calculate_metrics(payload['y_real'].reshape(payload['y_real'].shape[0]), np.ravel(payload['y_pred']))

    temp['NAME'] = name
    temp['INPUT SIZE'] = in_size
    temp['OUTPUT SIZE'] = out_size
    temp['TARGET'] = target
    temp['MAPE'] = payload['MAPE']
    temp['SMAPE'] = payload['sMAPE']
    temp['MAE'] = payload['MAE']
    temp['MDA'] = payload['MDA']

    results.append(temp)
    i = i + 1

ml_df = pd.DataFrame(results)


In [ ]:
ml_df[ml_df['NAME'] == 'bagging'].sort_values(['INPUT SIZE'], ascending=True).reset_index(drop=True)